In [1]:
import warnings
import pandas as pd
import numpy as np
import sys
import os
warnings.filterwarnings('ignore')
from Sessional_api import Sessional as se
from datetime import datetime
from datetime import date
from tqdm import tqdm
from sqlserver_api import syntax
from HSX_Transform_api import edit_data


root_folder: d:\FTA_sourcecode\INFOBANK_module\HOSE_module
[WinError 183] Cannot create a file when that file already exists: 'd:\\FTA_sourcecode\\INFOBANK_module\\HOSE_module\\log_HOSE'


In [2]:
source_path = r"C:\Users\researchFPA\fpts.com.vn\FPTS_HN - INFOBANK (2)\Thongtintt\KQGD Hose"

In [3]:
def close(data,x):
        """ format Session.close từ 2007 - 2023
        
        Args:
            data: DataFrame raw sau khi pd.read
            
            x: Date dữ liệu trong tên của folder chứa dữ liệu (VD: folder dữ liệu HOSE 16-08-2023)

        Returns:
            datab: DataFrame đã qua xử lý
        """ 
        row = data[data.iloc[:,1].str.startswith(('A','B'), na=False)==True].index.values.astype(int)[0]
        data.columns = data.iloc[row-2]
        data = data[row:]
        num = 0
        for col in data.columns:
            if isinstance(col, float) and np.isnan(col):
                data.rename(columns={col: 'col' + str(num)}, inplace=True)
                num += 1
        if ('col0' in data.columns.to_list()) & ('col1' in data.columns.to_list()) & ('col2' not in data.columns.to_list()):
            data.rename(columns={'Stt\nNo.':'pcID','Mã CK\nStock code':'ticker',"Giá đóng cửa hôm trước\nPrior day's close":'priorDayClose',
                                'Đóng cửa \nToday close':'closePrice','Thay đổi\nChange':'atcChangeStatus','KLGD\nTrading volume':'atcTradingVolume',
                                'GTGD (ng.đ) \nTrading value (VND 1,000)':'atcTradingValue','Tổng KLGD \nTotal volume':'totalVolume',
                                'Tổng GTGD (ng.đ)\nTotal value (VND 1,000)':'totalValue','KL niêm yết\nNo. of listed shares':'listedShares','KL đang lưu hành \nOutstanding shares':'outstandingShares',
                                'KL đang lưu hành điều chỉnh \nAdjusted outstanding shares':'adjOutstandingShares','Giá trị vốn hóa thị trường (tỷ đ)\nMarket cap. (VNDbil.)':'marketCap',
                                'Thay đổi (*)\nChange':'atcChangeStatus','Giá đóng cửa hôm trước\nPrior closing price':'priorDayClose','KL niêm yết\nCurrent listing shares':'listedShares',
                                'col0':'atcChangePoint','col1':'atcChangePercent',},inplace=True)
        elif ('col0' in data.columns.to_list()) & ('col1' in data.columns.to_list()) & ('col2' in data.columns.to_list()):
            data.drop(columns='col0',inplace=True)
            data.rename(columns={'Stt\nNo.':'pcID','Mã CK\nStock code':'ticker',"Giá đóng cửa hôm trước\nPrior day's close":'priorDayClose',
                                'Đóng cửa \nToday close':'closePrice','Thay đổi\nChange':'atcChangeStatus','KLGD\nTrading volume':'atcTradingVolume',
                                'GTGD (ng.đ) \nTrading value (VND 1,000)':'atcTradingValue','Tổng KLGD \nTotal volume':'totalVolume',
                                'Tổng GTGD (ng.đ)\nTotal value (VND 1,000)':'totalValue','KL niêm yết\nNo. of listed shares':'listedShares','KL đang lưu hành \nOutstanding shares':'outstandingShares',
                                'KL đang lưu hành điều chỉnh \nAdjusted outstanding shares':'adjOutstandingShares','Giá trị vốn hóa thị trường (tỷ đ)\nMarket cap. (VNDbil.)':'marketCap',
                                'Thay đổi (*)\nChange':'atcChangeStatus','Giá đóng cửa hôm trước\nPrior closing price':'priorDayClose','KL niêm yết\nCurrent listing shares':'listedShares',
                                'col1':'atcChangePoint','col2':'atcChangePercent',},inplace=True)
        data = data[data['ticker'].map(lambda x : type(x) == str)]
        data.replace({'atcChangeStatus':{3:'-',4:'+',7:'--',8:'++','<':'o'}},inplace=True)
        data['report_date'] = x
        data['pcID'] = data['ticker'] + x.replace('-','')
        data['last_update'] = str(date.today())
        data['priorDayClose'] = data['priorDayClose'] *1000
        data['closePrice'] = data['closePrice'] *1000
        data['atcChangePoint'] = data['atcChangePoint'] *1000
        data['atcTradingValue'] = data['atcTradingValue'] *1000
        data['totalValue'] = data['totalValue'] *1000
        data['marketCap'] = data['marketCap'] *10**9
        return data

In [4]:
folder = []
name = []
sheet = []
full_name = []
dffull = pd.DataFrame()

for root, dirs, files in os.walk(source_path):
    for i in tqdm(files):
        if 'TKGD tung phien (Sessions)' in i:
            df = pd.read_excel(os.path.join(root,i), sheet_name='3')
            datedata = i.split("\\")[-1][0:8]
            if bool(datetime.strptime(datedata, '%Y%m%d')) == True:
                datestr = datetime.strptime(datedata, '%Y%m%d').strftime('%Y-%m-%d')
                df = close(df,datestr)
                dffull = pd.concat([dffull,df])
                
dffull['report_date'] = pd.DatetimeIndex(dffull['report_date'])
dffull.sort_values(by=['report_date','ticker'], inplace=True)
dffull.drop_duplicates(inplace=True)
dffull['report_date'] = dffull['report_date'].astype('str')
dffull = dffull.fillna(value=np.nan).replace([np.nan],[None])

dffull.reset_index(drop=True,inplace=True)

dffull

100%|██████████| 1/1 [00:00<?, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 8/8 [00:00<?, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 4/4 [00:00<?, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 8/8 [00:00<?, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 8/8 [00:00<?, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 9/9 [00:00<?, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 8/8 [00:00<00:00, 8010.13it/s]
0it [00:00, ?it/s]
100%|██████████| 10/10 [00:00<?, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 9/9 [00:00<?, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 9/9 [00:00<?, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 9/9 [00:00<?, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 9/9 [00:00<?, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 9/9 [00:00<?, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 9/9 [00:00<?, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 9/9 [00:00<?, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 9/9 [00:00<?, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 10/10 [00:00<?, ?it/s

,pcID,ticker,priorDayClose,closePrice,atcChangeStatus,atcChangePoint,atcChangePercent,atcTradingVolume,atcTradingValue,totalVolume,totalValue,listedShares,outstandingShares,adjOutstandingShares,marketCap,report_date,last_update
0,AAM20130104,AAM,23800.0,23800.0,o,0.0,0.0,0,0.000000e+00,0,0.000000e+00,11339864,8639864.0,8639864.0,205628763200.0,2013-01-04,2024-05-30
1,ABT20130104,ABT,42500.0,42300.0,-,200.0,0.004706,130,5.499000e+06,1450,6.158400e+07,13607207,11007207.0,11007207.0,465604856099.999939,2013-01-04,2024-05-30
2,ACC20130104,ACC,25700.0,25600.0,-,100.0,0.003891,80,2.048000e+06,12840,3.280580e+08,10000000,10000000.0,10000000.0,256000000000.0,2013-01-04,2024-05-30
3,ACL20130104,ACL,12100.0,12200.0,+,100.0,0.008264,2000,2.440000e+07,2720,3.320600e+07,18399675,18399675.0,18399675.0,224476035000.0,2013-01-04,2024-05-30
4,AGD20130104,AGD,64000.0,64000.0,o,0.0,0.0,0,0.000000e+00,0,0.000000e+00,11999930,11999930.0,11999930.0,767995520000.0,2013-01-04,2024-05-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1022055,VTB20240529,VTB,10500.0,10100.0,-,400.0,0.038095,1000,1.010000e+07,1800,1.856500e+07,11982050,10804520.0,10804520.0,109125652000.0,2024-05-29,2024-05-30
1022056,VTO20240529,VTO,13500.0,13150.0,-,350.0,0.025926,36800,4.839200e+08,1577100,2.110730e+10,79866666,79866666.0,79866666.0,1050246657900.0,2024-05-29,2024-05-30
1022057,VTP20240529,VTP,79400.0,78500.0,-,900.0,0.011335,44400,3.485400e+09,686600,5.452050e+10,121783042,121783042.0,121783042.0,9559968797000.0,2024-05-29,2024-05-30
1022058,YBM20240529,YBM,10100.0,10100.0,o,0.0,0.0,0,0.000000e+00,0,0.000000e+00,14299880,14299880.0,14299880.0,144428788000.0,2024-05-29,2024-05-30


In [5]:
query = """
        SELECT *
        FROM INFOBANK_FPA.HOSE.SessionPreclose_stock
        """
old_df = syntax.SQL_select_to_pd(query)
old_df['report_date'] = pd.to_datetime(old_df['report_date'])
old_df

,tranID,ticker,priorDayClose,closePrice,atcChangeStatus,atcChangePoint,atcChangePercent,atcTradingVolume,atcTradingValue,totalVolume,totalValue,listedShares,outstandingShares,adjOutstandingShares,marketCap,report_date,last_update
0,AAA20161125,AAA,0.0,29900.0,None,NaN,NaN,23600.0,705640000.0,594710.0,1.818269e+10,51899988.0,51899988.0,51899988.0,1.551810e+12,2016-11-25,2024-05-15
1,AAA20161128,AAA,29900.0,29400.0,-,500.0,0.016722,15030.0,441882000.0,172720.0,5.065828e+09,51899988.0,51899988.0,51899988.0,1.525860e+12,2016-11-28,2024-05-15
2,AAA20161129,AAA,29400.0,29400.0,o,0.0,0.000000,5000.0,147000000.0,184960.0,5.419802e+09,51899988.0,51899988.0,51899988.0,1.525860e+12,2016-11-29,2024-05-15
3,AAA20161130,AAA,29400.0,29400.0,o,0.0,0.000000,9000.0,264600000.0,270300.0,7.922018e+09,51899988.0,51899988.0,51899988.0,1.525860e+12,2016-11-30,2024-05-15
4,AAA20161201,AAA,29400.0,28200.0,-,1200.0,0.040816,20000.0,564000000.0,340430.0,9.825579e+09,51899988.0,51899988.0,51899988.0,1.463580e+12,2016-12-01,2024-05-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1316804,YEG20240517,YEG,9440.0,9400.0,-,40.0,0.004237,44300.0,416420000.0,1542800.0,1.461917e+10,131353264.0,131353264.0,131353264.0,1.234721e+12,2024-05-17,2024-05-24
1316805,YEG20240520,YEG,9400.0,9340.0,-,60.0,0.006383,83700.0,781758000.0,1065100.0,9.990956e+09,131353264.0,131353264.0,131353264.0,1.226839e+12,2024-05-20,2024-05-24
1316806,YEG20240521,YEG,9340.0,9400.0,+,60.0,0.006424,74700.0,702180000.0,2183000.0,2.011551e+10,131353264.0,131353264.0,131353264.0,1.234721e+12,2024-05-21,2024-05-24
1316807,YEG20240522,YEG,9400.0,9550.0,+,150.0,0.015957,80400.0,767820000.0,1612500.0,1.540341e+10,131353264.0,131353264.0,131353264.0,1.254424e+12,2024-05-22,2024-05-24


In [6]:
df_pull = dffull[~dffull['pcID'].isin(list(set(old_df['tranID'])))]
df_pull

,pcID,ticker,priorDayClose,closePrice,atcChangeStatus,atcChangePoint,atcChangePercent,atcTradingVolume,atcTradingValue,totalVolume,totalValue,listedShares,outstandingShares,adjOutstandingShares,marketCap,report_date,last_update
1020408,AAA20240524,AAA,11450.0,11000.0,-,450.0,0.039301,320700,3.527700e+09,13309100,1.507561e+11,382274496,382274496.0,382274496.0,4205019456000.0,2024-05-24,2024-05-30
1020409,AAM20240524,AAM,8800.0,8870.0,+,70.0,0.007955,200,1.774000e+06,6500,5.730600e+07,12346411,10451182.0,10451182.0,92701984340.0,2024-05-24,2024-05-30
1020410,AAT20240524,AAT,4700.0,4650.0,-,50.0,0.010638,22100,1.027650e+08,243400,1.123902e+09,70819103,70819103.0,70819103.0,329308828950.0,2024-05-24,2024-05-30
1020411,ABR20240524,ABR,12850.0,13600.0,+,750.0,0.058366,1600,2.176000e+07,23200,2.978450e+08,20000000,20000000.0,20000000.0,272000000000.0,2024-05-24,2024-05-30
1020412,ABS20240524,ABS,5390.0,5260.0,-,130.0,0.024119,38300,2.014580e+08,469200,2.502717e+09,80000000,80000000.0,80000000.0,420800000000.0,2024-05-24,2024-05-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1022055,VTB20240529,VTB,10500.0,10100.0,-,400.0,0.038095,1000,1.010000e+07,1800,1.856500e+07,11982050,10804520.0,10804520.0,109125652000.0,2024-05-29,2024-05-30
1022056,VTO20240529,VTO,13500.0,13150.0,-,350.0,0.025926,36800,4.839200e+08,1577100,2.110730e+10,79866666,79866666.0,79866666.0,1050246657900.0,2024-05-29,2024-05-30
1022057,VTP20240529,VTP,79400.0,78500.0,-,900.0,0.011335,44400,3.485400e+09,686600,5.452050e+10,121783042,121783042.0,121783042.0,9559968797000.0,2024-05-29,2024-05-30
1022058,YBM20240529,YBM,10100.0,10100.0,o,0.0,0.0,0,0.000000e+00,0,0.000000e+00,14299880,14299880.0,14299880.0,144428788000.0,2024-05-29,2024-05-30


In [7]:
syntax.push('INFOBANK_FPA.HOSE.SessionPreclose_stock',df_pull)
syntax.SQL_close()

1652it [00:18, 91.07it/s] 

Push data Done
Connect Close
